# Steam Australian Users Data 

- `data/australian_user_reviews.json.gz`
- `data/australian_users_items.json.gz`


In [ ]:
import gzip
import ast
from pathlib import Path
import pandas as pd

DATA_DIR = Path('data')

def load_python_dicts_gz(path: Path, max_rows=None, verbose=True) -> pd.DataFrame:
    """Load a .json.gz file where each line is a Python dict literal.

    Many McAuley Steam datasets are stored as one Python dict per line, using
    single quotes instead of strict JSON. This helper parses them safely using
    ast.literal_eval, then flattens into a pandas DataFrame.

    Parameters
    ----------
    path : Path
        Path to the .json.gz file.
    max_rows : int or None
        If not None, stop after reading this many lines (useful for quick
        exploration). Set to None to load the full file.
    verbose : bool
        If True, print progress every 100k lines.
    """
    rows = []
    with gzip.open(path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            rows.append(ast.literal_eval(line))
            if max_rows is not None and len(rows) >= max_rows:
                break
            if verbose and i % 100_000 == 0:
                print(f"Read {i} lines from {path.name}...")

    df = pd.json_normalize(rows)
    return df


## australian_user_reviews.json.gz

Load a sample of the reviews file and inspect its columns.

In [2]:
reviews_path = DATA_DIR / 'australian_user_reviews.json.gz'
reviews = load_python_dicts_gz(reviews_path, max_rows=100_000)
print('reviews shape:', reviews.shape)
print('reviews columns:')
print(list(reviews.columns))
reviews.head()

reviews shape: (25799, 3)
reviews columns:
['user_id', 'user_url', 'reviews']


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


## australian_users_items.json.gz

Load a sample of the items file and inspect its columns.

In [3]:
items_path = DATA_DIR / 'australian_users_items.json.gz'
items = load_python_dicts_gz(items_path, max_rows=100)
print('items shape:', items.shape)
print('items columns:')
print(list(items.columns))
items.head()

items shape: (100, 5)
items columns:
['user_id', 'items_count', 'steam_id', 'user_url', 'items']


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [4]:
cell = reviews['reviews'].iloc[0]
print(type(cell))
cell

<class 'list'>


[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [5]:
first_review = cell[0]
first_review

{'funny': '',
 'posted': 'Posted November 5, 2011.',
 'last_edited': '',
 'item_id': '1250',
 'helpful': 'No ratings yet',
 'recommend': True,
 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}

In [6]:
from pprint import pprint
pprint(first_review)

{'funny': '',
 'helpful': 'No ratings yet',
 'item_id': '1250',
 'last_edited': '',
 'posted': 'Posted November 5, 2011.',
 'recommend': True,
 'review': 'Simple yet with great replayability. In my opinion does "zombie" '
           'hordes and team work better than left 4 dead plus has a global '
           'leveling system. Alot of down to earth "zombie" splattering fun '
           'for the whole family. Amazed this sort of FPS is so rare.'}


In [7]:
cell_items = items['items'].iloc[0]
print(type(cell_items))
cell_items

<class 'list'>


[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

In [8]:
first_item = cell_items[0]
from pprint import pprint
pprint(first_item)

{'item_id': '10',
 'item_name': 'Counter-Strike',
 'playtime_2weeks': 0,
 'playtime_forever': 6}


In [10]:
import pandas as pd

# 'reviews' DataFrame
# columns: ['user_id', 'user_url', 'reviews']

# explode the list so each review gets its own row
reviews_long = reviews.explode("reviews").reset_index(drop=True)

# split the dict in 'reviews' into separate columns
review_details = pd.json_normalize(reviews_long["reviews"])

reviews_long = pd.concat(
    [reviews_long.drop(columns=["reviews"]), review_details],
    axis=1
)

reviews_long.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [11]:
# starting from your 'items' DataFrame
# columns: ['user_id', 'items_count', 'steam_id', 'user_url', 'items']

items_long = items.explode("items").reset_index(drop=True)

item_details = pd.json_normalize(items_long["items"])
items_long = pd.concat(
    [items_long.drop(columns=["items"]), item_details],
    axis=1
)

items_long.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0


In [12]:
print("reviews_long shape:", reviews_long.shape)
print("items_long shape:", items_long.shape)

print("Unique users in reviews:", reviews_long["user_id"].nunique())
print("Unique users in items:",   items_long["user_id"].nunique())
print("Unique items (reviews):", reviews_long["item_id"].nunique())
print("Unique items (items):",   items_long["item_id"].nunique())

reviews_long shape: (59333, 9)
items_long shape: (10158, 8)
Unique users in reviews: 25485
Unique users in items: 100
Unique items (reviews): 3682
Unique items (items): 2758


In [13]:
reviews_per_user = reviews_long.groupby("user_id")["item_id"].nunique()
reviews_per_user.describe()

count    25485.00000
mean         2.29276
std          2.00138
min          0.00000
25%          1.00000
50%          1.00000
75%          3.00000
max         10.00000
Name: item_id, dtype: float64

In [14]:
owned_per_user = items_long.groupby("user_id")["item_id"].nunique()
owned_per_user.describe()

count    100.00000
mean     101.50000
std      139.67831
min        0.00000
25%       23.75000
50%       59.00000
75%      116.50000
max      888.00000
Name: item_id, dtype: float64

In [15]:
items_long["playtime_forever"].describe()

count     10150.000000
mean       1244.261379
std        7692.834643
min           0.000000
25%           0.000000
50%          56.000000
75%         374.750000
max      303466.000000
Name: playtime_forever, dtype: float64

In [16]:
reviews_long["recommend"].value_counts(normalize=True)

recommend
True     0.884799
False    0.115201
Name: proportion, dtype: float64

In [17]:
# keep only rows with an item_id
reviews_long = reviews_long.dropna(subset=["item_id"])

# parse 'posted' like "Posted November 5, 2011."
posted_clean = (
    reviews_long["posted"]
      .str.replace("Posted ", "", regex=False)
      .str.rstrip(".")
)

reviews_long["timestamp"] = pd.to_datetime(posted_clean, errors="coerce")

# drop rows we couldn't parse
reviews_long = reviews_long.dropna(subset=["timestamp"])

# sort by time per user
reviews_long = reviews_long.sort_values(["user_id", "timestamp"])
reviews_long.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,timestamp
21131,--000--,http://steamcommunity.com/id/--000--,,"Posted March 20, 2014.",,1250,1 of 1 people (100%) found this review helpful,True,หนุกคับ แนะนำ 10/10,2014-03-20
58236,--ace--,http://steamcommunity.com/id/--ace--,,"Posted January 24, 2014.",,440,2 of 3 people (67%) found this review helpful,True,the best game i ever plllayed,2014-01-24
58237,--ace--,http://steamcommunity.com/id/--ace--,,"Posted May 30, 2014.",,113200,0 of 1 people (0%) found this review helpful,True,One Of The Funnyest Games That Is Animated :) ...,2014-05-30
32158,--ionex--,http://steamcommunity.com/id/--ionex--,,"Posted August 15, 2015.",,105600,No ratings yet,True,"It's an amazing game, and im glad that mac use...",2015-08-15
45374,-2SV-vuLB-Kg,http://steamcommunity.com/id/-2SV-vuLB-Kg,,"Posted October 7, 2014.",,440,1 of 1 people (100%) found this review helpful,True,the best part of the Team Fortress 2,2014-10-07


In [21]:
reviews[['user_id', 'user_url']].head(10)

,user_id,user_url
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,js41637,http://steamcommunity.com/id/js41637
2,evcentric,http://steamcommunity.com/id/evcentric
3,doctr,http://steamcommunity.com/id/doctr
4,maplemage,http://steamcommunity.com/id/maplemage
5,Wackky,http://steamcommunity.com/id/Wackky
6,76561198079601835,http://steamcommunity.com/profiles/76561198079...
7,MeaTCompany,http://steamcommunity.com/id/MeaTCompany
8,76561198089393905,http://steamcommunity.com/profiles/76561198089...
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...


In [29]:
mask = reviews['user_id'].astype(str).str.contains('--', na=False)

weird_users = (
    reviews.loc[mask, ['user_id', 'user_url']]
           .drop_duplicates()
           .sort_values('user_id')
)

weird_users


,user_id,user_url
8342,--000--,http://steamcommunity.com/id/--000--
25239,--ace--,http://steamcommunity.com/id/--ace--
12954,--ionex--,http://steamcommunity.com/id/--ionex--
14302,Meph--,http://steamcommunity.com/id/Meph--


In [18]:
# make sure item_id is the same type in both tables
reviews_long["item_id"] = reviews_long["item_id"].astype(str)
items_long["item_id"]   = items_long["item_id"].astype(str)

interactions = reviews_long.merge(
    items_long[["user_id", "item_id", "item_name", "playtime_forever", "playtime_2weeks"]],
    on=["user_id", "item_id"],
    how="left"
)

interactions.head()


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,timestamp,item_name,playtime_forever,playtime_2weeks
0,--000--,http://steamcommunity.com/id/--000--,,"Posted March 20, 2014.",,1250,1 of 1 people (100%) found this review helpful,True,หนุกคับ แนะนำ 10/10,2014-03-20,NaN,NaN,NaN
1,--ace--,http://steamcommunity.com/id/--ace--,,"Posted January 24, 2014.",,440,2 of 3 people (67%) found this review helpful,True,the best game i ever plllayed,2014-01-24,NaN,NaN,NaN
2,--ace--,http://steamcommunity.com/id/--ace--,,"Posted May 30, 2014.",,113200,0 of 1 people (0%) found this review helpful,True,One Of The Funnyest Games That Is Animated :) ...,2014-05-30,NaN,NaN,NaN
3,--ionex--,http://steamcommunity.com/id/--ionex--,,"Posted August 15, 2015.",,105600,No ratings yet,True,"It's an amazing game, and im glad that mac use...",2015-08-15,NaN,NaN,NaN
4,-2SV-vuLB-Kg,http://steamcommunity.com/id/-2SV-vuLB-Kg,,"Posted October 7, 2014.",,440,1 of 1 people (100%) found this review helpful,True,the best part of the Team Fortress 2,2014-10-07,NaN,NaN,NaN


### ***Are recommended games the ones with higher playtime?***

**As shown below:**

- Games the user recommends tend to have much higher playtime than games they don’t recommend.
- There are very few non-recommended games (4), so that side is unreliable.
- The recommended group has at least one extremely large outlier (243,840), which makes the mean huge; the median and quartiles (589, 1,972, 9,299) are better summaries.

In [20]:
(
    interactions.groupby("recommend")["playtime_forever"]
    .describe()
)


,count,mean,std,min,25%,50%,75%,max
recommend,,,,,,,,
False,4.0,208.250000,287.461737,22.0,46.75,88.0,249.5,635.0
True,71.0,11389.760563,30797.988253,20.0,589.50,1972.0,9299.0,243840.0
